In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import SimpleImputer
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import RobustScaler,  StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile('data_usage_production.parquet')

for i in parquet_file.iter_batches(batch_size=10000):
    df = i.to_pandas()
    break

In [6]:
df

,telephone_number,tariff_desc,customer_status,tenure,data_compl_usg_local_m2,data_amount_lte_m2,data_pack_usg_m2,dpi_https_and_default_m2,data_pack_rev_local_m2,refill_total_m2,...,data_tariff_revenue_m2,data_from_tariff_m3,data_tariff_revenue_m3,data_from_tariff_m4,data_tariff_revenue_m4,data_from_tariff_m5,data_tariff_revenue_m5,data_from_tariff_m6,data_tariff_revenue_m6,data_compl_usg_local_m1
0,bCu4ClcqPl,Bolge,Active,473,33.0,33.0,4.0,1.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,944.22
1,K2rxyvPEu5,Azercellim,Suspended,1836,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,4XR-19v5Zy,SuperSen 1GB,Suspended,73,8.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.58
3,v7naSlyc_r,Azercellim,Active,2456,144.0,144.0,107.0,0.0,0.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154.92
4,X2vZ1E7oa6,SuperSen 20GB,Active,2632,15315.0,15243.0,451.0,1690.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15724.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,QCORIItErT,Bolge,Suspended,3426,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
9996,fQrS9JLRM1,SuperSen 10GB,Suspended,1154,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
9997,8oPqTsDmnu,Azercellim,Suspended,421,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.07
9998,yyiFvGAa8l,Bolge,Active,3410,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [10]:
df['tariff_desc'].value_counts()

tariff_desc
Bolge                     3607
Azercellim                2238
SuperSen 1GB               749
SuperSen 3GB               444
G6                         425
GencSim                    358
IsteSen                    348
G9                         320
G2                         310
SuperSen 6GB               203
Sade 6 Tariff              175
Bizimkiler                 144
Her Yere (Bizden-Bize)     138
SuperSen 10GB              129
SuperSen 20GB              115
SuperSen 30GB               49
DataCard                    45
Serbest25                   41
Serbest15                   38
Serbest6                    33
Serbest10                   32
Premium 50GB                27
Tourist Welcome             17
Serbest75                    3
Premium 75GB                 3
Tourist Super                2
Serbest50                    2
Premium 100GB                2
Unknown                      2
Tourist MAX                  1
Name: count, dtype: int64

The ***target column*** is **"data_compl_usg_local_m1"**,  the ***index column*** must be **"telephone_number"** - the hashcodes for the real telephone numbers. 
 
There are 67 features, 1 - index (telephone_number) and target "data_compl_usg_local_m1". Please do the feature engineering with pipelines and columntransformer. Use the simple estimator for training as RandomForrestRegressor https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html. Remember the train/test data splitting must be done at the beginning. 

In [8]:
df = df.set_index('telephone_number')

In [10]:
df.head()

,tariff_desc,customer_status,tenure,data_compl_usg_local_m2,data_amount_lte_m2,data_pack_usg_m2,dpi_https_and_default_m2,data_pack_rev_local_m2,refill_total_m2,dpi_tik_tok_m2,...,data_tariff_revenue_m2,data_from_tariff_m3,data_tariff_revenue_m3,data_from_tariff_m4,data_tariff_revenue_m4,data_from_tariff_m5,data_tariff_revenue_m5,data_from_tariff_m6,data_tariff_revenue_m6,data_compl_usg_local_m1
telephone_number,,,,,,,,,,,,,,,,,,,,,
bCu4ClcqPl,Bolge,Active,473,33.0,33.0,4.0,1.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,944.22
K2rxyvPEu5,Azercellim,Suspended,1836,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4XR-19v5Zy,SuperSen 1GB,Suspended,73,8.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.58
v7naSlyc_r,Azercellim,Active,2456,144.0,144.0,107.0,0.0,0.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154.92
X2vZ1E7oa6,SuperSen 20GB,Active,2632,15315.0,15243.0,451.0,1690.0,0.0,30.0,389.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15724.98


In [12]:
X = df.drop("data_compl_usg_local_m1", axis = 1)
y = df["data_compl_usg_local_m1"]

In [14]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(include='object').columns

In [20]:
cat_cols

Index(['tariff_desc', 'customer_status', 'lasttariff_m2', 'lasttariff_m3',
       'lasttariff_m4', 'lasttariff_m5', 'lasttariff_m6'],
      dtype='object')

In [22]:
df['tariff_desc'].value_counts()

tariff_desc
Bolge                     3607
Azercellim                2238
SuperSen 1GB               749
SuperSen 3GB               444
G6                         425
GencSim                    358
IsteSen                    348
G9                         320
G2                         310
SuperSen 6GB               203
Sade 6 Tariff              175
Bizimkiler                 144
Her Yere (Bizden-Bize)     138
SuperSen 10GB              129
SuperSen 20GB              115
SuperSen 30GB               49
DataCard                    45
Serbest25                   41
Serbest15                   38
Serbest6                    33
Serbest10                   32
Premium 50GB                27
Tourist Welcome             17
Serbest75                    3
Premium 75GB                 3
Tourist Super                2
Serbest50                    2
Premium 100GB                2
Unknown                      2
Tourist MAX                  1
Name: count, dtype: int64

In [24]:
df['customer_status'].value_counts()

customer_status
Active       5577
Suspended    4423
Name: count, dtype: int64

In [26]:
df['lasttariff_m2'].value_counts()

lasttariff_m2
Bolge                     3624
Azercellim                2261
SuperSen 1GB               749
SuperSen 3GB               429
G6                         424
GencSim                    358
IsteSen                    343
G9                         316
G2                         313
SuperSen 6GB               202
Sade 6 Tariff              171
Bizimkiler                 144
Her Yere (Bizden-Bize)     140
SuperSen 10GB              125
SuperSen 20GB              105
SuperSen 30GB               47
DataCard                    45
Serbest25                   43
Serbest15                   40
Serbest10                   35
Serbest6                    34
Premium 50GB                21
Tourist Welcome             17
Serbest75                    3
Tourist Super                2
Serbest50                    2
Premium 75GB                 2
Premium 100GB                2
Tourist MAX                  1
G5                           1
Preactive Sade Tariff        1
Name: count, dtype: int64

In [28]:
nominal_features = ['tariff_desc', 'customer_status', 'lasttariff_m2', 'lasttariff_m3',
       'lasttariff_m4', 'lasttariff_m5', 'lasttariff_m6']
numerical_features = num_cols.tolist()

In [30]:
pipe_num = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler())
])

pipe_nom = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('catboost', CatBoostEncoder(handle_unknown='ignore'))
])



preprocessor = ColumnTransformer([
    ('numerical', pipe_num, numerical_features),
    ('nominal', pipe_nom, nominal_features)
])

model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)


C:\Users\HUAWEI\anaconda3\Lib\site-packages\category_encoders\cat_boost.py:180: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp = y.groupby(X[col].astype(str)).agg(['cumsum', 'cumcount'])
C:\Users\HUAWEI\anaconda3\Lib\site-packages\category_encoders\cat_boost.py:180: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp = y.groupby(X[col].astype(str)).agg(['cumsum', 'cumcount'])
C:\Users\HUAWEI\anaconda3\Lib\site-packages\category_encoders\cat_boost.py:180: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Da

In [34]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")


Mean Squared Error (MSE): 13123070.3913
Mean Absolute Error (MAE): 1103.5846
R^2 Score: 0.7451
